In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

files = ["Dataset/Equities/AAPL.csv", "Dataset/Equities/META.csv", "Dataset/Equities/ORCL.csv", "Dataset/Equities/PEP.csv"]
dfs = [pd.read_csv(file) for file in files]

for df in dfs:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

combined_df = pd.concat(dfs, axis=0)
combined_df.sort_index(inplace=True)

In [5]:
def create_features(df):
    df['Returns'] = df['Close'].pct_change()
    df['Volatility'] = df['Returns'].rolling(window=10).std()
    df['Momentum'] = df['Close'] / df['Close'].shift(10) - 1
    df.dropna(inplace=True)
    return df

combined_df = create_features(combined_df)
features = combined_df[['Returns', 'Volatility', 'Momentum']]
labels = combined_df['Close']

scaler = StandardScaler()
features = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=False)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [8]:
class OptimizationLayer(nn.Module):
    def __init__(self):
        super(OptimizationLayer, self).__init__()

    def forward(self, pred, target):
        profit = torch.sum(pred - target)
        return profit
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.opt_layer = OptimizationLayer()
        
    def forward(self, x, target):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        profit = self.opt_layer(x, target)

        return x, profit

model = Net()

def custom_loss_function(pred, target, profit):
    loss = -profit
    return loss

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs, profit = model(X_train, y_train)
    loss = custom_loss_function(outputs, y_train, profit)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    test_outputs, test_profit = model(X_test, y_test)
    test_loss = custom_loss_function(test_outputs, y_test, test_profit)
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [10/100], Loss: 824235.2500
Epoch [20/100], Loss: 816513.4375
Epoch [30/100], Loss: 805360.8750
Epoch [40/100], Loss: 789550.8750
Epoch [50/100], Loss: 767698.3750
Epoch [60/100], Loss: 738271.8125
Epoch [70/100], Loss: 699577.7500
Epoch [80/100], Loss: 649775.1250
Epoch [90/100], Loss: 586889.7500
Epoch [100/100], Loss: 508818.6562
Test Loss: 283217.0000
